Мы будем работать с Postgres, так что надо проинсталлировать библиотеку для работы с ним.

In [1]:
#!sudo apt install libpq5=13.4-0ubuntu0.21.04.1
#!sudo apt install libpq-dev
!pip3 install psycopg2

In [2]:
# Подключаем библиотеки.
import psycopg2

from lxml import html
import requests

Сперва надо создать соединение с сервером, не забыв указать где находится сервер, с какой базой данных мы собираемся работать, логин и пароль для пользователя.

In [3]:
con = psycopg2.connect(host="localhost", port=65432, database="postgres", user="postgres", password="mysecretpassword")

Далее создаем курсор для отправки запросов и получения результатов.

In [4]:
cur = con.cursor()

Выполним несложный запрос при помощи функции execute - выбрать все поля и все записи из таблицы articles. 

SELECT отвечает за выборку данных. * означает, что мы выбираем все поля, но здесь можно написать конкретные поля через запятую. Поля могут содержать имя таблицы, например, articles.id_article. После FROM идут таблицы, из которых делается выборка (не забываем указать схему).

In [5]:
cur.execute("SELECT * FROM lenta_ru.articles")

Далее надо выбрать данные из курсора. Курсор не сможет отправить следующий запрос, если в нем остались не выбранные данные.

In [6]:
res = cur.fetchall()

In [7]:
res

[(12,
  '«Люди не признают других машин» Как тайно попавшие в\xa0СССР японские автомобили захватили восток России? ',
  datetime.date(2021, 11, 1),
  'Мир узнал о дрифте в 2006 году, после выхода на экраны фильма «Тройной форсаж: Токийский дрифт». Зрители со всего света были шокированы этим таинственным японским искусством. Вот только мало кто знал, что к тому времени им владели не только жители Страны восходящего солнца, но и бывалые стритрейсеры Владивостока. Всему виной\xa0— уникальный автомобильный мир, за много лет до этого возникший на Дальнем Востоке, мир со своими легендами, героями, языком, традициями и особым колоритом. Еще во времена Советского Союза сюда тайно поставляли первые иномарки сотрудники торгового флота. С тех пор машина для жителя Дальнего Востока\xa0— больше чем просто средство передвижения. Ничего удивительного, что новый гоночный стиль и вся JDM-культура, переведенная на русский как «культура правого руля», осели именно тут. Не мосты, не архитектура, а японски

Итак, у нас получилось. Хотя это из-за того, что я уже что-то положил в таблицу.

Теперь посмотрим как можно добавлять записи при помощи INSERT INTO.

In [8]:
cur.execute("INSERT INTO lenta_ru.articles (title, date, article, id_author, abstract) VALUES ('bla-bla-bla', date(now()),'bla333-bla-bla',0,'John Doe'), ('bla-bla-bla', date(now()),'bla333-bla-bla333',0,'John Doe')")

После изменений данных необходимо подтвердить при при помощи функции соединения `commit` или откатить назад при помощи `rollback()`. Если изменения не подтвердить, СУБД откатит их автоматически по истечении таймаута.

In [9]:
con.commit()

In [10]:
con.rollback()

Теперь загрузим статьи с сайта lenta.ru и положим их в базу.

In [11]:
# Класс, хранящий информацию о статье.
class LentaArticle:
    def __init__(self):
        self.date = ""
        self.author = ""
        self.head = ""
        self.text = ""
        self.abstract = ""
    
def getArticleLenta(adr: str) -> LentaArticle:
    ''' Загружает с сайта lenta.ru все статью, задаваемую http-адресом adr.
    '''
    page = requests.get(adr)
    #print(r.text)
    art = LentaArticle()
    
    tree = html.fromstring(page.text)
    art.head = tree.xpath(".//h1")[0].text_content()
#     art.date = tree.xpath(".//div[@class='b-topic__info']//time")[0].text_content().strip()
    art.date = '.'.join(adr.split('/')[4:7])
    art.author = tree.xpath(".//span[@itemprop='name']")[0].text_content().strip()
    try:
        art.abstract = tree.xpath(".//meta[@itemprop='description']")[0].get("content")
    except:
        art.abstract = ''

    art.text = '\n'.join([p.text_content() for p in tree.xpath(".//div[@itemprop='articleBody']/p")])
    
    return art

def getDayArticles(adr: str) -> list[LentaArticle]:
    ''' Загружает с сайта lenta.ru все статьи за день, задаваемый http-адресом adr.
    '''
    articles = []
    day = requests.get(adr)
    tree = html.fromstring(day.text)
    body = tree.xpath(".//a[@class='titles']")
    links = ['https://lenta.ru' + x.attrib["href"] for x in body]
    for link in links[:10]: # идем по первым 10 ссылкам на новости за день.
        print(link)
        art = getArticleLenta(link)
        if art.text != '':
            articles.append(art)
            
    return articles

def insertLentaArticle(art: LentaArticle, cur, con):
    ''' Функция кладет статью в базу данных.
        Предварительно проверяется наличие автора статьи в таблице authors. 
        Если его там нет, он кладется в таблицу.
    '''
    # Ищем автора и его id.
    cur.execute(f"SELECT id_author FROM lenta_ru.authors WHERE name='{art.author}'")
    authors = cur.fetchall()
    if len(authors) == 0: # Если автора нет - добавим его.
        cur.execute(f"INSERT INTO lenta_ru.authors (name, comment) VALUES ('{art.author}', '')")
        con.commit()
        cur.execute(f"SELECT id_author FROM lenta_ru.authors a WHERE a.name='{art.author}'")
        authors = cur.fetchall()
        
    id_author = authors[0][0]
    # Заменим апострофы, которые являются специальным символом для SQL на \'
    head = art.head.replace("'", "\\'")
    text = art.text.replace("'", "\\'")
    abstract = art.abstract.replace("'", "\\'")
    
    req = f"INSERT INTO lenta_ru.articles (title, date, article, id_author, abstract) VALUES " \
          f"('{head}', date('{art.date}'), '{text}', {id_author}, '{abstract}')"
    cur.execute(req)
    con.commit()
    

Достанем статьи с сайта.

In [12]:
arts = getDayArticles("https://lenta.ru/2021/11/01/")

https://lenta.ru/articles/2021/11/01/rf1/
https://lenta.ru/articles/2021/11/01/noveseries/
https://lenta.ru/news/2021/11/01/bastrykin_tuva/
https://lenta.ru/news/2021/11/01/advice/
https://lenta.ru/news/2021/10/31/record/
https://lenta.ru/news/2021/11/01/medvedev_europe/
https://lenta.ru/news/2021/11/01/diplomacy/
https://lenta.ru/news/2021/11/01/kadyrov/
https://lenta.ru/news/2021/11/01/stavki_vklady/
https://lenta.ru/news/2021/11/01/endocrinologist/


Посмотрим что вообще загружается.

In [13]:
arts[0].head

'«Люди не признают других машин» Как тайно попавшие в\xa0СССР японские автомобили захватили восток России? '

In [14]:
arts[0].abstract

''

In [15]:
arts[0].author

'Сергей Лютых'

In [16]:
for art in arts:
    insertLentaArticle(art, cur, con)

### Краткий экскурс в скачивание сайтов

`requests` - библиотека для отправки http-запросов.  
`lxml` - библиотека для работы с XML и HTML.

**XPath**  
[Ссылка 1](https://habr.com/ru/post/526774/)  
[Ссылка 2](https://habr.com/ru/post/464897/)

XPath позволяет задать шаблон для пути от корня XML-дерева к интересующей нас вершине.

- . - корень XML-дерева
- / - переход на один уровень ниже.
- // - переход на ноль или больше уровней вниз.
- \* - любая вершина.
- xyz - название вершины.
- [@feature] - вершина с параметром feature.
- [@feature='111'] - вершина с параметром feature, равным "111".
- xyz[n] - n-ый потомок вершины xyz.

### Пару слов о безопасности

![](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

Вот так гораздо лучше.

```Python
req = f"INSERT INTO lenta_ru.articles (title, date, article, id_author, abstract) VALUES " \
      f"(%s, date(%s), %s, %d, %s)"
cur.execute(req, (head, art.date, text, id_author, abstract))
con.commit()
```

Заодно не надо будет заменять апострофы (и кое-что ещё).

In [28]:
def insertLentaArticle2(art: LentaArticle, cur, con):
    ''' Функция кладет статью в базу данных.
        Предварительно проверяется наличие автора статьи в таблице authors. 
        Если его там нет, он кладется в таблицу.
    '''
    # Ищем автора и его id.
    cur.execute(f"SELECT id_author FROM lenta_ru.authors WHERE name='{art.author}'")
    authors = cur.fetchall()
    if len(authors) == 0: # Если автора нет - добавим его.
        cur.execute(f"INSERT INTO lenta_ru.authors (name, comment) VALUES ('{art.author}', '')")
        con.commit()
        cur.execute(f"SELECT id_author FROM lenta_ru.authors a WHERE a.name='{art.author}'")
        authors = cur.fetchall()
        
    id_author = authors[0][0]

    req = f"INSERT INTO lenta_ru.articles (title, date, article, id_author, abstract) VALUES " \
          f"(%s, date(%s), %s, %s, %s)"
    cur.execute(req, (art.head, art.date, art.text, id_author, art.abstract))
    con.commit()

In [31]:
for art in arts:
    print(art.head)
    insertLentaArticle2(art, cur, con)

«Люди не признают других машин» Как тайно попавшие в СССР японские автомобили захватили восток России? 
Возвращение великих. «Король тигров», Декстер и шальная Екатерина II в главных сериалах ноября
Бастрыкин взял под контроль расследование убийства двух сестер в Туве
Байден дал совет России по добыче нефти
Байден объяснил рекордное падение своих рейтингов
Медведев обвинил Европу в «пещерной логике» во время пандемии
На Украине признали провалы в дипломатии
Кадыров рассказал о поощрении для победивших на турнире UFC бойцов «Ахмата»
Ставки по вкладам в России превысили восемь процентов годовых
Эндокринолог перечислил способы восполнить дефицит витамина D


In [27]:
len(arts)

10